## Problem Set 4

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset4_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing

In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset4_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [2]:
## your code here
debar = pd.read_csv("pset4_inputdata/debar.csv")
debar.head()

,Name,"City, State",Violation,Start date,viol_num
0,Macky and Brad Farms,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1
1,Labatte Farms,"Froid, MT",Impeding the Audit Process – Non- Response,1/17/2016,viol1
2,"RB Farm Picking, Inc","Miami, FL",Failure to respond to audit (no response),10/26/2013,viol1
3,Abe- Nancy Froese,"Seminole, TX",Failure to respond to audit request,11/23/2017,viol1
4,Quentin Schell,"Towner, North Dakota",Non-payment of certification fee,7/22/2016,viol1


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [3]:
## your code here
#number of rows in debar and the number of unique employer names
print("There are ",debar.shape[0], " rows and ", debar.Name.nunique(), " unique employer names.\n")
print("Since the number of rows exceed the number of unique employer names, we can assume that there is more than one row per employers")

There are  105  rows and  94  unique employer names.

Since the number of rows exceed the number of unique employer names, we can assume that there is more than one row per employers


## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [4]:
## your code here
#A
debar["is_repeated"]= [True if debar.Name.value_counts()[name] != 1 else False for name in debar.Name]

#B
#By printing the dataframe when debar.is_repeated is true, 
#we can know that some firms violate the regulation several times one or several days
#or commit one violation throughout several occasions.
debar[debar.is_repeated].sort_values("Name")

#C
mult_bar = debar[debar.is_repeated].sort_values("Name")
mult_bar.head()
mult_bar.shape

,Name,"City, State",Violation,Start date,viol_num,is_repeated
9,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,7/6/2014,viol2,True
95,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),7/6/2014,viol1,True
70,Cisco Produce Inc.,"Cairo, GA",Failure to respond to audit (no response),12/10/2014,viol2,True
49,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,12/10/2015,viol1,True
102,Dove Creek Farms,"Mount Vernon, TX",Failure to respond to audit request,2/9/2018,viol2,True
24,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2/9/2018,viol1,True
74,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),12/10/2014,viol2,True
48,F&W Farms,"Ingalls, KS",Impeding the Audit Process – Partial- Response,12/10/2014,viol1,True
0,Macky and Brad Farms,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1,True
14,Macky and Brad Farms,"Plains, TX",Failure to respond to audit (no response),2/13/2015,viol2,True


,Name,"City, State",Violation,Start date,viol_num,is_repeated
9,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,7/6/2014,viol2,True
95,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),7/6/2014,viol1,True
70,Cisco Produce Inc.,"Cairo, GA",Failure to respond to audit (no response),12/10/2014,viol2,True
49,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,12/10/2015,viol1,True
102,Dove Creek Farms,"Mount Vernon, TX",Failure to respond to audit request,2/9/2018,viol2,True


(22, 6)

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [5]:
## your code here

mult_debar_wide = pd.pivot(mult_bar, index = ["Name", "City, State"], columns = "viol_num", values = "Start date").reset_index()
mult_debar_wide.columns = ["start_date_"+column if "viol" in column else column for column in mult_debar_wide.columns]

mult_debar_wide.head()
mult_debar_wide.shape

,Name,"City, State",start_date_viol1,start_date_viol2
0,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014
1,Cisco Produce Inc.,"Cairo, GA",12/10/2015,12/10/2014
2,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018
3,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014
4,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015


(11, 4)

## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [6]:
## your code here

#A
#creating a "is_dup" column that is True when start_date_viol1 and start_date_viol2 are equal.
mult_debar_wide["is_dup"] = [True if mult_debar_wide.start_date_viol1[index] == mult_debar_wide.start_date_viol2[index] else False for index in range(mult_debar_wide.shape[0])]
mult_debar_wide

#B
#list of employers that has a duplicate
dups = mult_debar_wide.Name[mult_debar_wide.is_dup]

#create a keep column that has False only when 1) the employer has duplicated record and 2) it is the second violation(viol2)
debar["keep"] = [False if (sum(dups.isin([debar["Name"][index]]))>0) & (debar.viol_num[index] == "viol2") else True\
                 for index in range(debar.shape[0])]
debar_1 = debar[debar.keep]
debar_clean = debar_1.drop(columns = ["is_repeated", "keep"])

debar_clean.shape
print("There are ", debar_clean.shape[0]," rows and ", debar_clean.Name.nunique(), " unqiue employer names")


,Name,"City, State",start_date_viol1,start_date_viol2,is_dup
0,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014,True
1,Cisco Produce Inc.,"Cairo, GA",12/10/2015,12/10/2014,False
2,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018,True
3,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014,True
4,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015,True
5,Mark Duncan,"Roosevelt, UT",11/16/2014,11/16/2014,True
6,Old Tree Farms/Verpaalen Custom Service,"Volga, SD",12/11/2014,12/1/2014,False
7,Rollo Farm Labor Contractor,"Miami, FL",8/23/2014,8/23/2014,True
8,SRT Farms,"Morton, TX",11/16/2014,11/16/2014,True
9,Sharon Mathis,"Tifton, GA",11/16/2014,11/16/2014,True


(97, 5)

There are  97  rows and  94  unqiue employer names


In [7]:
debar_clean.reset_index()

,index,Name,"City, State",Violation,Start date,viol_num
0,0,Macky and Brad Farms,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1
1,1,Labatte Farms,"Froid, MT",Impeding the Audit Process – Non- Response,1/17/2016,viol1
2,2,"RB Farm Picking, Inc","Miami, FL",Failure to respond to audit (no response),10/26/2013,viol1
3,3,Abe- Nancy Froese,"Seminole, TX",Failure to respond to audit request,11/23/2017,viol1
4,4,Quentin Schell,"Towner, North Dakota",Non-payment of certification fee,7/22/2016,viol1
...,...,...,...,...,...,...
92,98,Ruben Ruiz (DeSoto Harvesting),"Arcadia, FL",WHD referral,10/8/2016,viol1
93,99,J & L Farms,"Claremont, SD",WHD referral,5/8/2018,viol1
94,101,Donna Lucio,"Lyons, Georgia",Non-payment of certification fee,8/25/2016,viol1
95,103,Trinity Livestock Partnership,"Earth, TX",Impeding the Audit Process – Non- Response,12/30/2015,viol1


# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [8]:
# your code here to load the data 
jobs = pd.read_csv("pset4_inputdata/jobs_clean.csv")

jobs.head()

,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1
0,H-300-20199-721302,"Fazio Farms Operating Company, LLC",Portland,OR,8433 NE 13th Ave
1,H-300-20231-773906,Charlie Sunderland,McMinnville,TN,982 Northcutts Cove Road
2,H-300-20231-774123,Michael Rudebusch,Livingston,TX,7366 State Hwy 146 S
3,H-300-20231-774151,Lodahl Farms,Brockton,MT,4198 Road 2049
4,H-300-20231-774508,"Dunson Harvesting, Inc.",Winter Haven,FL,400 Eagle Lake Loop Rd.


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [9]:
## your code here

#Rafael Barajas is the only exact match between the two dataset

pd.merge(jobs, debar_clean, how = "inner", left_on = "EMPLOYER_NAME", right_on = "Name")

,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1,Name,"City, State",Violation,Start date,viol_num
0,H-300-20287-876656,Rafael Barajas,Port St. Lucie,FL,2598 SW Dalpina Rd.,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,9/23/2016,viol1


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [10]:
## insert your code to turn into uppercase here

#A 
#Convert to uppercase using list comprehension

Upper_employer_name=[name.upper() for name in jobs.EMPLOYER_NAME]
Upper_name = [name.upper() for name in debar_clean.Name]


In [11]:
## insert your code for the random sample

#Print random sample
from random import sample
random.sample(Upper_employer_name, 15)
random.sample(Upper_name, 15)

['WESTERN RANGE ASSOCIATION',
 'THOMAS HONEY FARM, INC.',
 'STEPHEN G MYERS',
 'THORNOCK RANCH LLC',
 'KUTIKS NUCS & QUEENS',
 'INGALLS HONEY, INC.',
 'WIGGINS FARMS, LLC',
 'FC OF ARCADIA, INC.',
 'JULIAN LAND & LIVESTOCK',
 'PAUGH, INC. ',
 'COASTAL FARM LABOR SERVICES, INC.',
 'JOSE H. MARTINEZ',
 'LEVIN SAVOY FARM/CAJUN CLASSIC CRAWFISH',
 'HANNAN DESHOTELS FARMS',
 'CATCH-22 TRUCKING LLC']

['CISCO PRODUCE INC.',
 'TRIANGLE J FARMS',
 'GONZALO FERNANDEZ*',
 'SLASH E.V. RANCH LLP',
 'DANECO, LLC',
 'ROLLO FARM LABOR CONTRACTOR',
 'JESUS LEDESMA',
 'SHARON MATHIS',
 'OLD TREE FARMS/VERPAALEN CUSTOM SERVICE',
 'DELIA ROJAS',
 '69 FARMS LLC',
 'AGECY II, LLC',
 'PROMAX INC.',
 'WALKER PLACE',
 'GERMAN FARMS']

In [12]:
## insert your code for assigning the uppercase names back to the data
jobs["EMPLOYER_NAME"] = Upper_employer_name
debar_clean["Name"] = Upper_name


### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [13]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [14]:
## insert your code here with the regex pattern for part A
pattern = r"(INC|CO|LLP)\."

## insert your code to use re.sub to apply the pattern to the test cases for part B
re.sub(pattern, r"\1", pos_example_1)
re.sub(pattern, r"\1", pos_example_2)
re.sub(pattern, r"\1", neg_example)

'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [15]:
## your code here to clean the columns
debar_clean["name_clean"]= [re.sub(pattern, r"\1", name) for name in debar_clean.Name]
jobs["name_clean"] = [re.sub(pattern, r"\1", name) for name in jobs.EMPLOYER_NAME]

In [16]:
## your code here to print the head
debar_clean.head(10)
jobs.head()

,Name,"City, State",Violation,Start date,viol_num,name_clean
0,MACKY AND BRAD FARMS,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1,MACKY AND BRAD FARMS
1,LABATTE FARMS,"Froid, MT",Impeding the Audit Process – Non- Response,1/17/2016,viol1,LABATTE FARMS
2,"RB FARM PICKING, INC","Miami, FL",Failure to respond to audit (no response),10/26/2013,viol1,"RB FARM PICKING, INC"
3,ABE- NANCY FROESE,"Seminole, TX",Failure to respond to audit request,11/23/2017,viol1,ABE- NANCY FROESE
4,QUENTIN SCHELL,"Towner, North Dakota",Non-payment of certification fee,7/22/2016,viol1,QUENTIN SCHELL
5,CHRIS ROBINSON,"Paint Lick, KY",Impeding the Audit Process – Non- Response,6/19/2015,viol1,CHRIS ROBINSON
6,"HAITI/USA WORKFORCE, LLC","Miami, FL",Impeding the Audit Process – Non- Response,6/19/2015,viol1,"HAITI/USA WORKFORCE, LLC"
7,TRUST NURSERY,"Pulaski, NY",Failure to respond to audit (partial response),3/21/2014,viol1,TRUST NURSERY
8,GONZALO FERNANDEZ*,"Watsonville, California",WHD referral,8/25/2016,viol1,GONZALO FERNANDEZ*
10,YOLANDA CHAVEZ FARMING,"Santa Maria, CA",Non-Payment,7/23/2014,viol1,YOLANDA CHAVEZ FARMING


,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1,name_clean
0,H-300-20199-721302,"FAZIO FARMS OPERATING COMPANY, LLC",Portland,OR,8433 NE 13th Ave,"FAZIO FARMS OPERATING COMPANY, LLC"
1,H-300-20231-773906,CHARLIE SUNDERLAND,McMinnville,TN,982 Northcutts Cove Road,CHARLIE SUNDERLAND
2,H-300-20231-774123,MICHAEL RUDEBUSCH,Livingston,TX,7366 State Hwy 146 S,MICHAEL RUDEBUSCH
3,H-300-20231-774151,LODAHL FARMS,Brockton,MT,4198 Road 2049,LODAHL FARMS
4,H-300-20231-774508,"DUNSON HARVESTING, INC.",Winter Haven,FL,400 Eagle Lake Loop Rd.,"DUNSON HARVESTING, INC"


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate the fuzzy matching we'll cover in future week or optional OH; that fuzzy matching would recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical

In [17]:
## your code here
#A 
#Inner merging debar_clean and jobs by name_clean
pd.merge(jobs, debar_clean, on = "name_clean")
#It did not reveal more matched employers 

#B
#creating "name_clean_2" to replace SLASH E.V. RANCH LLP to SLASH EV RANCH 
regex = r"(\w+\s\w)\.(\w)\.(\s\w+)\s.+"

debar_clean["name_clean_2"]=["".join((re.match(regex, name).group(1), re.match(regex, name).group(2), re.match(regex, name).group(3))) \
       if bool(re.match(regex, name)) else name \
       for name in debar_clean.name_clean]

debar_clean[[bool(re.match(regex, name)) for name in debar_clean.name_clean]]

#C
#Conducint a left join using "name_clean_2"
pd.merge(jobs, debar_clean, left_on = "name_clean", right_on = "name_clean_2", suffixes= ("_jobs", "_debar"))

,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1,name_clean,Name,"City, State",Violation,Start date,viol_num
0,H-300-20287-876656,RAFAEL BARAJAS,Port St. Lucie,FL,2598 SW Dalpina Rd.,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,9/23/2016,viol1


,Name,"City, State",Violation,Start date,viol_num,name_clean,name_clean_2
41,SLASH E.V. RANCH LLP,"Rifle, CO",WHD Debarment,11/15/2014,viol1,SLASH E.V. RANCH LLP,SLASH EV RANCH


,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1,name_clean_jobs,Name,"City, State",Violation,Start date,viol_num,name_clean_debar,name_clean_2
0,H-300-20287-876656,RAFAEL BARAJAS,Port St. Lucie,FL,2598 SW Dalpina Rd.,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,9/23/2016,viol1,RAFAEL BARAJAS,RAFAEL BARAJAS
1,H-300-20306-894148,SLASH EV RANCH,Rifle,CO,22593 County Road 5,SLASH EV RANCH,SLASH E.V. RANCH LLP,"Rifle, CO",WHD Debarment,11/15/2014,viol1,SLASH E.V. RANCH LLP,SLASH EV RANCH


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [18]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern
rg = r"(.+)\s\(COMPANY\)\sAND\s(.+)\s\(INDIVIDUAL\)"

## your code here to apply it to the pos_example
re.findall(rg, pos_example)

## your code here to apply it to the negative example
re.findall(rg, neg_example)


[('COUNTY FAIR FARM', 'ANDREW WILLIAMSON')]

[]

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [19]:
# your code here

debar_clean["co_name"] = [re.match(rg, name).group(1) if bool(re.match(rg, name)) else "" for name in debar_clean.name_clean]
debar_clean["ind_name"] = [re.match(rg, name).group(2) if bool(re.match(rg, name)) else "" for name in debar_clean.name_clean]



   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [24]:
# your code here
debar_clean[["name_clean", "co_name", "ind_name", "Violation"]]\
[(debar_clean["name_clean"]== "CISCO PRODUCE INC" ) | (debar_clean["ind_name"]== "ANDREW WILLIAMSON")]

,name_clean,co_name,ind_name,Violation
49,CISCO PRODUCE INC,,,Impeding the Audit Process – Non- Response
70,CISCO PRODUCE INC,,,Failure to respond to audit (no response)
84,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM,ANDREW WILLIAMSON,WHD Debarment


# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [ ]:
## your code here